In [ ]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.transformations.advanced_transformations import EnumerateStructureTransformation
from pymatgen.transformations.standard_transformations import OrderDisorderedStructureTransformation, SubstitutionTransformation

from pymatgen.io.vasp.sets import batch_write_input, MPRelaxSet
from pymatgen.analysis.energy_models import EwaldElectrostaticModel
from pymatgen.analysis.bond_valence import BVAnalyzer

s1 = Structure.from_file("NbH4-Pn-3m-170GPa.cif")
data = {"Nb":+4, "H":-1}
# analyzer = SpacegroupAnalyzer(s1)
# prim_cell = analyzer.get_primitive_standard_structure()

s1.add_oxidation_state_by_element(data)
# print(s1)
# enum = EnumerateStructureTransformation()
# enumerated = enum.apply_transformation(prim_cell, 100)


species_map = {"H":{"X":0.0546875, "H":0.9453125}}
substitution = SubstitutionTransformation(species_map)
s2 = substitution.apply_transformation(s1)
# print(s2)


s2.make_supercell(2) # 可以通过structure的make_supercell()方法直接建 
# print(s2)
order = OrderDisorderedStructureTransformation() 
standard_structures = order.apply_transformation(s2, return_ranked_list=100) # 排序，这里我们将return_ranked_list设置为100以获得100个结构

# structures = [d["structure"] for d in enumerated]
# print("%d structures returned." % len(structures))
# from pymatgen.io.vasp.inputs import Poscar
# n = 0
# for Cry_Str in structures:
#     #print(Cry_Str)
#     Vasp_Str = Poscar(Cry_Str)
#     n = n+1
#     Vasp_Str.write_file('%d.vasp'%n)
# from pymatgen.io.cif import CifWriter

#n = 0
#for Cry_Str in structures:
#    n = n + 1
#    cif_filename = f"{n}.cif"
#    cif_writer = CifWriter(Cry_Str)
#    cif_writer.write_file(cif_filename)   

#energy= [d["energy"] for d in enumerated]
#fl = open('energy.dat','w')
#fl.writelines(str(energy))
#data = {"K":1, "O":-2, "Fe":2,"Ti":4.25,"Mn":3.75}
#abc=[]
#for i in range(1,101):
#    m = Structure.from_file('%d.vasp'%i)
#    m.add_oxidation_state_by_element(data)
#    energy = EwaldElectrostaticModel().get_energy(m)
#    abc.append(energy)
#f2 = open('energy2.dat','w')
#f2.writelines(str(abc))
# with open('energy.dat', 'w') as f:
#     for item in abc:
#         f.write(str(item) + '\n')

#strcturelist2.sort(key = EwaldElectrostaticModel().get_energy)

In [1]:
from pymatgen.core import Structure, Lattice
from pymatgen.analysis.energy_models import EwaldSummation
from pymatgen.transformations.standard_transformations import SupercellTransformation
from pymatgen.io.vasp import Poscar

import itertools
import numpy as np

from multiprocessing import Pool, cpu_count

s1 = Structure.from_file("NbH4-Pn-3m-170GPa.cif")
data = {"Nb":+4, "H":-1}###???????????????????????????


sc = SupercellTransformation().from_scaling_factors(1,1,1)
s2 = sc.apply_transformation(s1)

# 找到所有氢原子的位置
h_indices = [i for i, site in enumerate(s2) if site.species_string == "H"]

# 计算需要移除的氢原子数量
num_h_atoms_to_remove = int(len(h_indices) * 0.25)
print(num_h_atoms_to_remove)

# 生成所有可能的排列组合，删除氢原子
combinations = itertools.combinations(h_indices, num_h_atoms_to_remove)

# # 定义一个列表来存储结果
results  = []
lowest10 = []

# # 遍历每一种组合，计算静电能
for combination in combinations:
    s3 = s2.copy()
    s3.remove_sites(combination)
    s3.add_oxidation_state_by_element(data)
    energy = EwaldSummation(s3).total_energy
    # print(energy)
    results.append((combination, energy))
    if len(lowest10) < 10:
        lowest10.append([s3, energy])
    else:
        max_energy = max(lowest10, key=lambda x: x[1])[1]
        if energy < max_energy:
            for i in range(len(lowest10)):
                if np.isclose(lowest10[i][1], max_energy):
                    lowest10[i] = [s3, energy]
                    Poscar(s3).write_file('POSCAR_'+str(i))
                    break

lowest10.sort(key=lambda x: x[1])

for idx, (s, energy) in enumerate(lowest10):
    print(energy)

e:\conda\envs\cage\lib\site-packages\pymatgen\io\cif.py:1229: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


4
-492.18719494897795
-492.18719494897795
-492.1871949489779
-492.1871949489779
-492.1871949489779
-492.1871949489779
-492.1871949489779
-492.1871949489779
-491.6591639290055
-491.6591639290055


In [41]:
import itertools
import numpy as np
from pymatgen.core import Structure
from pymatgen.transformations.standard_transformations import SupercellTransformation
from pymatgen.analysis.ewald import EwaldSummation
from pymatgen.io.vasp import Poscar
from multiprocessing import Pool, cpu_count

# 读取结构文件
s1 = Structure.from_file("NbH4-Pn-3m-170GPa.cif")
data = {"Nb": +4, "H": -1}

# 创建超级胞
sc = SupercellTransformation().from_scaling_factors(1, 1, 1)
s2 = sc.apply_transformation(s1)

# 找到所有氢原子的位置
h_indices = [i for i, site in enumerate(s2) if site.species_string == "H"]

# 计算需要移除的氢原子数量
num_h_atoms_to_remove = int(len(h_indices) * 0.25)
print(num_h_atoms_to_remove)

# 生成所有可能的排列组合，删除氢原子
combinations = list(itertools.combinations(h_indices, num_h_atoms_to_remove))

def calculate_energy(combination):
    s3 = s2.copy()
    s3.remove_sites(combination)
    s3.add_oxidation_state_by_element(data)
    energy = EwaldSummation(s3).total_energy
    return combination, energy, s3

if __name__ == "__main__":
    # 创建一个进程池
    with Pool(cpu_count()) as pool: # cpu_count() 函数：返回当前系统中 CPU 核心的数量
        # 并行计算每个组合的能量
        results = pool.map(calculate_energy, combinations) # Pool 类：创建一个进程池，其中的进程数通常设置为 CPU 核心数，但你可以根据需要进行调整。

    # 找到能量最低的10种组合
    lowest10 = sorted(results, key=lambda x: x[1])[:10]

    # 保存结果并输出能量
    for idx, (combination, energy, s3) in enumerate(lowest10):
        Poscar(s3).write_file('POSCAR_' + str(idx))
        print(energy)


e:\conda\envs\cage\lib\site-packages\pymatgen\io\cif.py:1229: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


4


In [2]:
import itertools
import numpy as np
from pymatgen.core import Structure
from pymatgen.transformations.standard_transformations import SupercellTransformation
from pymatgen.analysis.ewald import EwaldSummation
from pymatgen.io.vasp import Poscar
from multiprocessing import Pool, cpu_count

# 读取结构文件
s1 = Structure.from_file("NbH4-Pn-3m-170GPa.cif")
data = {"Nb": +4, "H": -1}

# 创建超级胞
sc = SupercellTransformation().from_scaling_factors(1, 1, 1)
s2 = sc.apply_transformation(s1)

# 找到所有氢原子的位置
h_indices = [i for i, site in enumerate(s2) if site.species_string == "H"]

# 计算需要移除的氢原子数量
num_h_atoms_to_remove = int(len(h_indices) * 0.25)
print(num_h_atoms_to_remove)

# 生成所有可能的排列组合，删除氢原子
combinations = list(itertools.combinations(h_indices, num_h_atoms_to_remove))

def calculate_energy(combination):
    s3 = s2.copy()
    s3.remove_sites(combination)
    s3.add_oxidation_state_by_element(data)
    energy = EwaldSummation(s3).total_energy
    return combination, energy, s3

def parallel_computations(combinations, num_workers):
    # 创建一个进程池
    with Pool(num_workers) as pool:
        # 并行计算每个组合的能量
        results = pool.map(calculate_energy, combinations)
    return results

if __name__ == "__main__":
    # 获取 CPU 核心数量，设置合理的进程数量
    num_cores = max(1, cpu_count() - 1)  # 保留一个核心给系统使用
    print(f"Number of CPU cores used: {num_cores}")

    # 批量处理，减少任务调度和通信开销
    batch_size = len(combinations) // num_cores
    print(f"batch_size: {batch_size}")
    batched_combinations = [combinations[i:i + batch_size] for i in range(0, len(combinations), batch_size)]

    results = []
    for batch in batched_combinations:
        results.extend(parallel_computations(batch, num_cores))

    # 找到能量最低的10种组合
    lowest10 = sorted(results, key=lambda x: x[1])[:10]

    # 保存结果并输出能量
    for idx, (combination, energy, s3) in enumerate(lowest10):
        Poscar(s3).write_file('POSCAR_' + str(idx))
        print(energy)


e:\conda\envs\cage\lib\site-packages\pymatgen\io\cif.py:1229: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


4
Number of CPU cores used: 11
batch_size: 165
